In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
#from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_neo4j import Neo4jGraph
#from langchain_community.graphs import Neo4jGraph
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

from dotenv import load_dotenv

load_dotenv()

In [ ]:
graph=Neo4jGraph()

In [ ]:
loader=PyPDFLoader('./data/test.pdf')
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=250)
documents=text_splitter.split_documents(docs)

In [ ]:
len(documents)

In [ ]:
llm_type=os.getenv('LLM_TYPE', 'ollama')
if llm_type=='openai':
    llm=ChatOpenAI(model='gpt-4o-mini', temperature=0)
else:
    llm=ChatOllama(model='llama3.1:8b', temperature=0)

llm_transformer=LLMGraphTransformer(llm)
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph_documents[0]

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
def showgraph():
    driver=GraphDatabase.driver(
        uri=os.environ['NEO4J_URI'],
        auth=(os.environ['NEO4J_USERNAME'], os.environ['NEO4J_PASSWORD']))
    session=driver.session()
    widget=GraphWidget(graph=session.run("Match (s)-[r:!MENTIONS]->(t) Return s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget
showgraph()